<a href="https://colab.research.google.com/github/Pranish-Pantha/Tensorflow-Google-Colabs/blob/master/Keras_V_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')
#training data
train_images2 = mnist.train.images
train_labels2 = mnist.train.labels
#test data
test_images2 = mnist.test.images
test_labels2 = mnist.test.labels
#categories
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
#normalization
train_images2 = train_images2/255.0
test_images2 = test_images2/255.0


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


This makes 4 sets of data: 2 for training the model and 2 for testing it.
It also makes the classifications written in an array. 
Finally, we normalize the data by dividing all the data by 255 (greyscale images are on a scale from 0 to 255)

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(),
     #keras.layers.InputLayer(input_shape= train_images2.shape, name="features"), this layer wasn't needed
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),
    ])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(train_images2, train_labels2, epochs=4)


Train on 55000 samples
Epoch 1/4
55000/55000 [==============================] - 3s 63us/sample - loss: 1.2901 - acc: 0.6883
Epoch 2/4
55000/55000 [==============================] - 3s 61us/sample - loss: 0.5349 - acc: 0.8666
Epoch 3/4
55000/55000 [==============================] - 3s 61us/sample - loss: 0.4096 - acc: 0.8904
Epoch 4/4
55000/55000 [==============================] - 3s 62us/sample - loss: 0.3615 - acc: 0.8995


This is where we make the model. It has a layer that flattens all the data into a usable shape, a hidden layer with a relu activiation function, and an output layer that uses softmax.
We compile the model here, which tells the model how it will back-propagate (using adam, based on the accuracy of categories).
Finally fitting the model actually trains it using the training data, with 4 epochs.

In [0]:
test_loss, test_acc = model.evaluate(test_images2, test_labels2)

#prediction = model.predict(test_images2) doesn't need to be done

print("Final loss: ",test_loss, "Final accuracy: ",test_acc)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.3335 - acc: 0.9065
Final loss:  0.33354910699129103 Final accuracy:  0.9065


This is where we use our test data to see how accurate the model is and what the loss is. We see the accuracy is about 90%

In [0]:
columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]
classifier = tf.estimator.DNNClassifier(
    feature_columns=columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir="./tmp/mnist_model"
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff735bc4048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


This makes the estimator, specifying optimizer, columns, hidden units, etc.

In [0]:
def getinput(dataset):
    return dataset.images, dataset.labels.astype(np.int32)


train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": getinput(mnist.train)[0]},
    y=getinput(mnist.train)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": getinput(mnist.test)[0]},
    y=getinput(mnist.test)[1],
    num_epochs=1,
    shuffle=False
)

These are the input functions that format the data in a way that the estimator can use

In [0]:
classifier.train(input_fn=train_input_fn, steps=1000) #more steps is usually better
effectiveness = classifier.evaluate(input_fn=test_input_fn) ["accuracy"]
print("Final accuracy: ",effectiveness)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_model/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5000 into ./tmp/mnist_model/model.ckpt.
INFO:tensorflow:loss = 3.158885, step = 5001
INFO:tensorflow:global_step/sec: 333.11
INFO:tensorflow:loss = 2.3881338, step = 5101 (0.301 sec)
INFO:tensorflow:global_step/sec: 397.026
INFO:tensorflow:loss = 8.265196, step = 5201 (0.252 sec)
INFO:tensorflow:global_step/sec: 407.827
INFO:tensorflow:loss = 15.957972, step = 5301 (0.245 sec)
INFO:tensorflow:global_step/sec: 402.554
INFO:tensorflow:loss = 3.6012728, step = 5401 (0.248 sec)
INFO:tensorflow:global_step/sec: 398.858
INFO:tensorflow:loss = 9.42451, step = 5501 (0.251 sec)
INFO:tensorflow:global_step/sec: 413.613
INFO:tensorflow:loss = 19.72007

This trains the model using the training function.
Then it evaluates the test data and prints out the accuracy, which is about 95%.

In [0]:
train_input_fn2 = tf.estimator.inputs.numpy_input_fn(
x={"input_1":getinput(mnist.train)[0] },  
y=getinput(mnist.train)[1],
batch_size=50,
num_epochs=None,
shuffle=True)

test_input_fn2 = tf.estimator.inputs.numpy_input_fn(
x={"input_1": getinput(mnist.test)[0]},  
y=getinput(mnist.test)[1],
num_epochs=1, #could have increased this, but it seemed unnecessary
shuffle=False)  


These are the new input functions because the dictionary for the converted model will be different

In [0]:

keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model)
keras_estimator.train(train_input_fn2, steps= 1000)
efficiency2 =keras_estimator.evaluate(input_fn=test_input_fn2)["acc"] #["accuracy"]had trouble getting this to work until I realized it was "acc"
print("Accuracy: ", effectiveness2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Note that this doesn't affect the state of the model instance you passed as `keras_model` argument.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_9zst1aj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff736e72e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_e

This is where the original keras model is converted into an estimator model. It is then trained and evaluated. The accuracy is about 93%.

The estimator api would be used for distributed environments and is able to rerun on multiple processing units.
In keras, I had to build a model with different layers. Estimator has pre-made estimators that I could choose from. Overall, however, both are high level tensorflow apis that make creating machine learning models easier.
For this particular dataset, the estimator model was more accurate, as seen with the first estimator model and the converted one.
